In [1]:
import csv
import pandas as pd
import numpy as np
import scipy.linalg as linalg
from hodgerank_tools import *

In [16]:
symptoms = ['A', 'B', 'C','D', 'E', 'F', 'G', 'H', 'I', 'J']
#covid_data = np.array([symptoms, [3, 2, 2,5], [7, 8, 9, np.nan], [2, 2, 1,3]])
covid_df = pd.DataFrame(np.array([[3, 2, 2,5, 2, 2,5, 2, 2,5], 
                                  [7, 8, 9, np.nan, 2, 2,5, 2, 2,5], 
                                  [2, 2, 1,3, 2, 2,5, 2, 2,5]]), columns=symptoms)

In [17]:
symptoms = ['A', 'B', 'C','D']
my_data = df_to_dict_list(covid_df)
print(my_data)

[{'A': 3.0, 'B': 2.0, 'C': 2.0, 'D': 5.0, 'E': 2.0, 'F': 2.0, 'G': 5.0, 'H': 2.0, 'I': 2.0, 'J': 5.0}, {'A': 7.0, 'B': 8.0, 'C': 9.0, 'E': 2.0, 'F': 2.0, 'G': 5.0, 'H': 2.0, 'I': 2.0, 'J': 5.0}, {'A': 2.0, 'B': 2.0, 'C': 1.0, 'D': 3.0, 'E': 2.0, 'F': 2.0, 'G': 5.0, 'H': 2.0, 'I': 2.0, 'J': 5.0}]


In [18]:
my_nodes = get_nodes(my_data)
print(my_nodes)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']


In [19]:
get_f_W(my_data, my_nodes)

(array([ 0. ,  0. ,  1.5, -2. , -2. ,  1. , -2. , -2. ,  1. ,  0. ,  2. ,
        -2. , -2. ,  1. , -2. , -2. ,  1. ,  2.5, -2. , -2. ,  1. , -2. ,
        -2. ,  1. , -2. , -2. ,  1. , -2. , -2. ,  1. ,  0. ,  3. ,  0. ,
         0. ,  3. ,  3. ,  0. ,  0. ,  3. , -3. , -3. ,  0. ,  0. ,  3. ,
         3. ]),
 array([[3., 0., 0., ..., 0., 0., 0.],
        [0., 3., 0., ..., 0., 0., 0.],
        [0., 0., 2., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 3., 0., 0.],
        [0., 0., 0., ..., 0., 3., 0.],
        [0., 0., 0., ..., 0., 0., 3.]]))

In [20]:
print(rank(my_data))

(  node         r
0    G  1.579310
1    J  1.579310
2    D  1.200000
3    A  0.475862
4    B  0.441379
5    C  0.406897
6    H -1.420690
7    E -1.420690
8    I -1.420690
9    F -1.420690, 7.731866825208086)


In [21]:
print(naive_rank_0(my_data))

  node    r
0    G  5.0
1    J  5.0
2    A  4.0
3    B  4.0
4    C  4.0
5    D  4.0
6    E  2.0
7    F  2.0
8    H  2.0
9    I  2.0


In [22]:
# Ranks by the mean pairwise difference of each node
def naive_rank(data):
    #initialize nodes, sum_scores, total_votes
    nodes = get_nodes(data)
    (f, W) = get_f_W(data, nodes)
    sum_scores = dict()
    naive_r = dict()
    for node in nodes:
        sum_scores[node] = 0
    total_votes = sum_scores.copy()
    edges = get_edges(nodes)
    for node in nodes:
        # populate total_votes, sum_scores
        for i, edge  in enumerate(edges):
            if edge[0] == node:
                sum_scores[node] -= f[i]
            if edge[1] == node:
                sum_scores[node] += f[i]
            else:
                continue
            total_votes[node] += W[i, i]
        # calculate final score
        if total_votes[node] == 0:
            naive_r[node] = 0
        else:
            naive_r[node] = sum_scores[node]/total_votes[node]
    # format into a df
    rank_df = pd.DataFrame.from_dict(naive_r, orient = 'index', columns = ['r'])
    rank_df = rank_df.reset_index()
    rank_df = rank_df.rename(columns = {'index':'node'})
    rank_df = rank_df.sort_values(by =['r'],  ascending = False)
    rank_df = rank_df.reset_index(drop = True)
    #(f, W) = get_f_W(data, nodes)
    return (rank_df) #get_error(f, W, naive_r, nodes)
naive_rank(my_data)

,node,r
0,D,2.000000
1,B,1.333333
2,G,0.941176
3,J,0.615385
4,C,0.583333
5,A,0.000000
6,I,-0.608696
7,H,-0.700000
8,F,-1.000000
9,E,-1.272727


In [23]:
print(group_similar_scoring(my_data, 3))

[['D', 'B', 'G', 'J'], ['C', 'A', 'I'], ['H', 'F', 'E']]


In [28]:
print(simple_group_rank(my_data, 3))

(  node         r
0    G  0.454545
1    J  0.454545
2    D      -0.0
3    B -0.909091
4    A  0.666667
5    C  0.666667
6    I -1.333333
7    E       0.0
8    F       0.0
9    H       0.0, 2.049586776859504)
